In [1]:
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine
import json
from django.core.serializers.json import DjangoJSONEncoder


from django.conf import settings
from django.db.models import Q


In [2]:
dbc = settings.LEGACY_DB_CONNECTION

In [3]:
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [4]:
table = "stichworte"

In [5]:
query = f"SELECT * FROM {table}"
df = pd.read_sql(query, con=db_connection)

In [6]:
df

,stid,stichwort,start,svar1,svar2,svar3,svar4,svar5,wurzel,anmerkung
0,1,abbas,Schlagwort,abba,abbatis,abbati,abbatem,abbate,abba,None
1,2,Afer,Eigenname,Afri,Afro,Afrum,Afros,,Afr,None
2,3,Africa,Eigenname,Africae,Africam,,,,Afric,None
3,4,Agarreni,Eigenname,Agareni,Aggareni,,,,Agaren,None
4,5,Alamannia,Eigenname,Alamanniae,Alamanniam,Alemannia,,,Alamanni,None
...,...,...,...,...,...,...,...,...,...,...
212,213,Wasco,Eigenname,Wascones,Wasconum,Vascon,,,Wasco,None
213,214,Wasconia,Eigenname,Wasconiae,Wasconiam,Vasconi,,,Wasconia,None
214,215,Wisigothi,Eigenname,Wisigothus,Wisigotha,Wisigothae,,,Wisigoth,None
215,216,gens gentium,Schlagwort,,,,,,,


In [7]:
for i, row in df.iterrows():
    item, _ = KeyWord.objects.get_or_create(legacy_pk=row['stid'])
    item.stichwort = row['stichwort']
    item.art = row['start']
    item.wurzel = row['wurzel']
    varianten = []
    for n in range(1,6):
        lookup = f"svar{n}"
        varianten.append(row[lookup].strip())
    item.varianten = ";".join(list(filter(None, varianten)))
    row_data = f"{json.dumps(row.to_dict(), cls=DjangoJSONEncoder)}"
    item.orig_data_csv = row_data
    item.save()

In [ ]:
varianten

In [ ]:
str_list = ";".join(list(filter(None, varianten)))

In [ ]:
str_list

In [ ]:
for i, row in df.head(20).iterrows():
    try:
        item = Stelle.objects.get(legacy_pk=row['stexttitel'])
        print(item.text)
    except:
        continue
    for st in stichwort_list:
        if row[st] != "":
            kw = row[st]
            keyws = KeyWord.objects.filter(
                Q(stichwort=kw)|
                Q(svar1=kw)
            )
    

In [ ]:
# run_import('archiv', file_class_map_dict=file_class_map_dict)

In [ ]:
KeyWord.objects.all().delete()